In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from math import ceil
import pickle

import numpy as np
import pandas as pd

In [2]:
datestring_old = '20191118'
datestring_new = '20200210'

In [4]:
with open('./Data/vyberove_konania_'+'20191119'+'.pkl', 'rb') as f:
    df_vk_old = pickle.load(f)

In [20]:
with open('./Data/df_preskocene_'+datestring_new+'.pkl', 'rb') as f:
    df_preskocene_new = pickle.load(f)
    
df_preskocene_new.index = df_preskocene_new.index.astype(int)

In [8]:
# legacy code: parse textoveho logu preskocenych konani do df
with open('./Data/log_preskocene_'+datestring_old+'.txt') as f:
    preskocene = f.readlines()

# zbehnutie kodu ak zlyha ukladanie niektorych datasetov
with open('./Data/log_preskocene_chyby_'+datestring_old+'.txt') as f:
    preskocene = preskocene + f.readlines()
    
# transformuj do formatu index - (priebežný) výsledok konania
def get_index_result(preskoceny_zaznam):
    preskoceny_zaznam_list = preskoceny_zaznam.split(' ')
    index = int(preskoceny_zaznam_list[1])
    vysledok = ' '.join(preskoceny_zaznam_list[3:]).strip('\n')
    
    return (index, vysledok)

preskocene_list = list(map(get_index_result, preskocene))
print(len(preskocene_list))

# poradime si s duplikatmi (opakovany zapis zaznamov, ktore sa kvoli chybe neulozili ale neskor sme ich znova zbehli)
df_preskocene_old = pd.DataFrame(pd.Series(dict(preskocene_list)))
df_preskocene_old.index.name = 'index_konania'
df_preskocene_old.rename(columns={0:'vysledok'}, inplace=True)

1546


In [22]:
preskocene_spolocne = np.intersect1d(df_preskocene_new.index, df_preskocene_old.index)

In [26]:
df = pd.DataFrame()

for ii in range(1, 25+1):
    with open('./Data/data_'+datestring_new+'_' + str(ii) + '.pkl', 'rb') as f:
        df0 = pickle.load(f)
    df = df.append(df0, ignore_index=True)

In [34]:
with open('./Data/rel_linky_list_'+datestring_old+'.pkl', 'rb') as f:
    relativne_linky_old = pickle.load(f)
    
with open('./Data/rel_linky_list_'+datestring_new+'.pkl', 'rb') as f:
    relativne_linky_new = pickle.load(f)

total_old = len(relativne_linky_old)
total_new = len(relativne_linky_new)

In [57]:
n_nove = total_new - total_old
n_total_nove_vyhodnotene = df.index.size
n_predtym_preskocene = df_preskocene_old.index.size - len(preskocene_spolocne)
n_teraz_preskocene = df_preskocene_new.index.size - len(preskocene_spolocne)

In [58]:
print(n_nove + n_predtym_preskocene)
print(n_teraz_preskocene + n_total_nove_vyhodnotene)

1907
1907


In [59]:
df_vk = df_vk_old.append(df, ignore_index=True)

In [64]:
print(df_vk.index.size + df_preskocene_new.index.size)
print(total_new)

21015
21015


In [65]:
# uloz
df_vk.to_excel('./Data/vyberove_konania_'+datestring_new+'.xlsx')
df_vk.to_pickle('./Data/vyberove_konania_'+datestring_new+'.pkl')